In [1]:
import numpy as np
import pandas as pd

In [2]:
train_agg = pd.read_csv("05d2b4ea-c-Dataset/train_data/AggregateData_Train.csv")
test_agg = pd.read_csv("05d2b4ea-c-Dataset/test_data/AggregateData_Test.csv")
print(test_agg.shape)
test_agg.head()

(48434, 53)


,UID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53
0,Te-1,61P4S1M3GET,10397.75,?,?,?,?,?,?,197186.68,...,?,?,5.04055,?,?,?,6,9,3,10
1,Te-2,6NYVS1J3U7M,?,606.74,?,?,?,?,?,?,...,?,?,?,?,?,?,1,?,?,?
2,Te-3,BLFB1KMS48,87452.89,963.52,?,?,13838,?,?,1081.25,...,1.28678,?,?,?,?,?,10,10,20,1
3,Te-4,MYGB1LCW8P,?,393.26,100000,?,?,?,?,?,...,?,?,?,?,?,?,4,1,?,2
4,Te-5,6UG5S1PQGE5,1100,497.2,?,?,?,?,0.9,18640,...,?,?,0.00470,?,?,0.00007,2,3,1,2


In [3]:
flag_dict = dict(zip(train_agg.V2, train_agg.bad_flag))
def get_pred(idx, pred_array):
    if test_agg.loc[idx, "V2"] in flag_dict:
        return flag_dict[test_agg.loc[idx, "V2"]]
    return pred_array[idx]

##### First Ensemble

In [4]:
pred = np.zeros(test_agg.shape[0])
count = 0

for i in range(5):
    cur_pred = np.load("predictions/rfc/test_%d.npy"%i)
    pred += cur_pred
    count += 1
    
for i in range(5):
    cur_pred = np.load("predictions/xgbc/test_%d.npy"%i)
    pred += cur_pred
    count += 1
    
pred /= count

In [11]:
submission = pd.DataFrame({"UID": test_agg.UID, "bad_flag": pred})
for i in submission.index:
    submission.loc[i, 'bad_flag'] = get_pred(i, pred)

In [12]:
submission.head()

,UID,bad_flag
0,Te-1,0.0
1,Te-2,0.0
2,Te-3,0.0
3,Te-4,0.0
4,Te-5,0.0


In [13]:
submission.to_csv("submissions/rfc_xgb.csv", index=False)